In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits  #displaying plots 
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib online
rcParams['figure.fisize'] = [10.,8.]

### read image

In [ ]:
#read image into a 2-D numpy array
data = fits.getdata("imagine.fits")

### show data

In [ ]:
#show the image
m,s = np.mean(data), np.std(data)
plt.imshow(data,interpolation='nearest',cmap='gray',vmin=m-s,vmax=m+s,origin='lower',)
plt.colorbar();
fig.savefig('plot.png')

### background subtraction

In [ ]:
#measure spatially varying background on image
bkg = sep.Background(data)

In [ ]:
bkg = sep.Background(data,mask=mask,bw=64,bh=64,fw=3,fh=3)

In [ ]:
#get global mean and noise of image's background
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate background as 2-D array but same size as original image
bkg_image = bkg.back()
#bkg_image = np.array(bkg)

In [ ]:
#show background
plt.imshow(bkg_image,interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();
fig.savefig('plot.pdf')

In [ ]:
#evaluate background noise as 2-D array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
#show background noise
plt.imshow(bkg_rms,interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();
fig.savefig('plot.pdf')

In [ ]:
#subtract background
data_sub = data - bkg

### object detection

In [ ]:
#set detection threshold to be a constant value of 1.5*sigma
#sigma=global background rms
objects = sep.extract(data_sub,1.5,err=bkg.globalrms)

In [ ]:
#number of objects detected
len(objects)

In [ ]:
#over-plot the object coordinates with some parameters on the image
#this will check where the detected objects are
from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
m,s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub,interpolation='nearest',cmap='gray',
               vmin=m-s,vmax=m+s,origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i]*180./np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

fig.savefig('plot.png')

In [ ]:
#see available fields
objects.dtype.names

### aperture photometry

In [ ]:
#perform circular aperture photometry 
#with a 3 pixel radius at locations of the objects
flux, fluxerr, flag = sep.sum_circle(data_sub,objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
#show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))
    